In [23]:
# load and process data
import os,gc,glob
# import sys
from importlib import reload
import numpy as np
# import matplotlib.pyplot as plt
from wbtool.show import wshow
from wbtool import file_misc as fm
import proc_data,aux

video_dir = '/home/wb/samba_dir/cells_brightfield/single-cell-movement-for-machine-learning/WB-F344'
# video_dir = '/home/wb/samba_dir/cells_brightfield/single-cell-movement-for-machine-learning/A549/'
# video_dir = '/home/wb/samba_dir/cells_brightfield/single-cell-movement-for-machine-learning/HELA/'
# '/home/wb/samba_dir/cells_brightfield/leadingcell-WB-entirespan/B1ROI1.npy'
# '/home/wb/samba_dir/cells_brightfield/leading-HELA-entirespan/A1ROI1.npy'
# vid = np.load('/home/wb/samba_dir/cells_brightfield/single-cell-movement-for-machine-learning/A549/0/A4ROI1.npy')
# vid = np.load('/home/wb/samba_dir/cells_brightfield/temp/A4ROI1.npy')  # difficult
annotation_dir = video_dir
cell_mask_dir = '/home/wb/samba_dir/cells_brightfield/model0808_mask/mask_WB-F344/' 
# cell_mask_dir = '/home/wb/samba_dir/cells_brightfield/model0808_mask/mask_A549/' 

# output masks
# output_dir = '/home/wb/samba_dir/indev/results' #io_args['output_mask_dir']
output_dir = '/home/wb/samba_dir/cells_brightfield/model0808_mask_trackres/WB-F344/' 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [33]:
'''
load video frames and point coordinates.
'''
# reload(proc_data)
dscode = 'A4ROI3'
fcode = 0

# vid = np.load(os.path.join(video_dir, '{}/{}.npy'.format(fcode, dscode)))
# # vid = vid[:5]
# vid = proc_data.value_modify(vid)
# vid = proc_data.ch_num_modify(vid)
# print(vid.dtype)

ids,coords = proc_data.read_coords_from_csv(os.path.join(annotation_dir, '{}/Results_{}.csv'.format(fcode, dscode)))

In [ ]:
# show one frame's image and point location. 用于确认载入的图像和point是否正确。
from wbtool import show 

fid = 1
# vid = seg_series
point_prompt = coords[fid] #[:,[1,0]]
# print(point_prompt)
obj_mode = np.ones((len(point_prompt), 1))
frame = vid[fid]
frame = proc_data.value_modify(frame)
print(frame.shape)

show.overlay_point(frame, point_prompt)

In [6]:
''' to get models and tracker needed for segmentation/tracking
'''
# import SegTracker
# from wbtool import use_models
# from model_args import aot_args,sam_args #,segtracker_args

# print(sam_args)
# sam_args['generator_args'] = {
#         'points_per_side': 30,
#         'pred_iou_thresh': 0.8,
#         'stability_score_thresh': 0.9,
#         'crop_n_layers': 1,
#         'crop_n_points_downscale_factor': 2,
#         'min_mask_region_area': 200,
#     }

segtracker_args = {
    'sam_gap': 1, # the interval to run sam to segment new objects
    'min_area': 20, # minimal mask area to add a new mask as a new object
    'max_obj_num': 255, # maximal object number to track in a video
    'min_new_obj_iou': 0.8, # the area of a new object in the background should > 80% 
    'new_replace_iou_min': 0.5,
    'remain_remove_max':0.4,
    'min_obj_area_ratio':0.0001,
    'max_obj_area_ratio':0.01
}

# tool models.
SA = use_models.cv.sam_agent('prompt')
segtracker = SegTracker.SegTracker(segtracker_args, sam_args, aot_args)

In [34]:
'''
get segmentation mask.
'''
# === get segmentation results of each frame.
# (this is independent of tracking activity)
# reload(aux)
use_saved = True
# file = os.path.join(output_dir,'../sms.pk')

if use_saved:
    # seg_series = fm.pload(file)
    # load multiple figures
    flist = glob.glob(os.path.join(cell_mask_dir, '{}/{}/*.png'.format(fcode, dscode)))
    flist.sort()
    print(len(flist))
    seg_series = []
    for file in flist:
        seg_series.append(fm.imread(file))
    # seg_series: 31 of [(904, 1224)] with dtype int32.
else:    
    seg_series = aux.do_segment(SA, segtracker_args, vid, ids, coords)
    fm.pdump(file, seg_series)

31


In [13]:
''' Generate Results for the Whole segmented series (不使用SegTracker，完全基于已分割的frame串起来的版本)
'''
from onevision import morphology as ovmorph
from onevision import morph_data
reload(morph_data)
use_point = False

fnum = len(seg_series)

pred_list = []; track_list = []

def link_map(M1, M2):
    ''' 在两个ID信息无关的map之间建立细胞间的对应关系。
    '''
    L = []
    for idx in M1:
        obj = M1[idx]
        tid = ovmorph.find_overlap(obj, M2, 'source')
        L.append([idx, tid])
    return L

# wshow(seg_series[1])
# wshow(seg_series[0])
link_L = []
for frame_idx in range(fnum-1):
    a = link_map(morph_data.IMbind(seg_series[frame_idx], 'map'), morph_data.IMbind(seg_series[frame_idx + 1], 'map'))
    link_L.append(a)
    # print("processed frame {}".format(frame_idx), end='\r')
# print(a)

In [35]:
'''
根据人工点的信息给对应的细胞mask设定id。
（不被点覆盖的mask忽略）
'''
# reload(aux)
new_series = aux.connect_id_mask(seg_series, ids, coords)

'''
post-processing 
'''
# reload(aux)
# === fix frame's masks.
for k in range(len(new_series)):
    new_series[k] = aux.fix_cell_mask(new_series[k], False, False)
    print('done', k, end='\r')

# === fix sequence.
from onevision import trajectory
nsms = aux.fix_trace(new_series, False)


{3: [2], 7: [4], 8: [5], 10: [22], 35: [21], 40: [27]}
{}
>> start single missing fix
{3: [2], 7: [4], 8: [5], 10: [22], 35: [21], 40: [27]}


In [36]:
# 保存细胞轨迹结果（以每帧的分割的结果的形式）
folder = os.path.join(output_dir, '{}/'.format(fcode))
print(folder)
# from misc_tool import show
# reload(show)
tp = os.path.join(folder, dscode)

# === 染色
# pm = proc_data.overlay_all(vid, nsms, show_id=True)
# proc_data.all2imgfile(pm, tp, dscode+'_', direct_write=True)

for k in range(len(nsms)):
    fm.pdump(os.path.join(tp, dscode+'_{}.pk'.format(k)), nsms[k].DF)

/home/wb/samba_dir/cells_brightfield/model0808_mask_trackres/WB-F344/0/


In [ ]:
''' Generate Results for the Whole Video (基于SegTracker的版本)
'''
# importlib.reload(SegTracker)
# from skimage import morphology
# from onevision import morphology as ovmorph
from onevision import morph_data
# from wbtool import show
# reload(aux)
# reload(proc_data)
# reload(morph_data)
# reload(ovmorph)

# grounding_caption = "cell"
# box_threshold, text_threshold, box_size_threshold, reset_image = 0.02, 0.02, 0.05, True

# === tracking
flag_do_backward = False
# segtracker = SegTracker.SegTracker(segtracker_args, sam_args, aot_args)
segtracker.restart_tracker()   # 调用的是aot_tracker/AOTTracker function, AOT模型自身的restart要求.
fnum = len(vid)

torch.cuda.empty_cache()
gc.collect()
pred_list = []; track_list = []; #sm2=[]; sm1=[]
# with torch.cuda.amp.autocast():
for frame_idx in range(fnum):
    frame = vid[frame_idx]
    track_list.append(None)
    if frame_idx == 0:
        # print(segtracker.curr_idx)
        pred_mask = morph_data.imbind_to_map(seg_series[frame_idx])
        segtracker.add_reference(frame, pred_mask)
    elif (frame_idx % segtracker_args['sam_gap']) == 0:
        # 1/2
        seg_mask = seg_series[frame_idx]
        # 2/2
        # seg_mask,_ = segtracker.detect_and_seg(frame, grounding_caption, box_threshold, text_threshold, box_size_threshold, reset_image)
        # e/2

        # torch.cuda.empty_cache()
        # gc.collect()
        track_mask = segtracker.track(frame)
        
        track_mask = morph_data.IMbind(track_mask, 'map')
        # print('track output', track_mask.ids)
        track_mask = aux.isolate_filter_object(track_mask, segtracker_args)
        track_list[frame_idx] = track_mask
        # print('k7') #('before st ', track_mask.ids)
        pred_mask = aux.merge_st(segtracker_args, track_mask, seg_mask)
        # print('k8') #('after st ', pred_mask)
        pred_mask = morph_data.imbind_to_map(pred_mask)
        # print('k10') #('merged ', list(np.unique(pred_mask)))
        
        # segtracker.restart_tracker()
        segtracker.add_reference(frame, pred_mask)
    else:
        pred_mask = segtracker.track(frame, update_memory=True)
    pred_list.append(pred_mask)
    # gc.collect()
    proc_data.save_prediction(pred_mask, output_dir, str(frame_idx)+'.png', opt='overlay', image=frame)    
    print("processed frame {}, obj_num {}".format(frame_idx, segtracker.get_obj_num()), end='\r')
print('\n-forward tracking finished')
fm.pdump(os.path.join(output_dir,'../tms.pk'), track_list)
fm.pdump(os.path.join(output_dir,'../pms.pk'), pred_list)

# == backward merge part.
if flag_do_backward:
    for frame_idx in range(fnum-2, -1, -1):
        print(frame_idx)
        frame = vid[frame_idx]

        if (frame_idx % segtracker_args['sam_gap']) == 0:
            # 1/2
            seg_mask = pred_list[frame_idx]
            # 2/2
            # seg_mask,_ = segtracker.detect_and_seg(frame, grounding_caption, box_threshold, text_threshold, box_size_threshold, reset_image)
            # e/2

            # torch.cuda.empty_cache()l
            # gc.collect()
            track_mask = segtracker.track(frame)
            track_mask = morph_data.IMbind(track_mask, 'map')
            track_mask = aux.isolate_filter_object(track_mask, segtracker_args)
            # track_list[frame_idx] = track_mask
            pred_mask = aux.merge_st(segtracker_args, track_mask, seg_mask)
            pred_mask = morph_data.imbind_to_map(pred_mask)

            # segtracker.restart_tracker()
            segtracker.add_reference(frame, pred_mask)
        else:
            pass

        proc_data.save_prediction(pred_mask, output_dir, str(frame_idx)+'.png', opt='overlay', image=frame)    
        print("processed frame {}, obj_num {}".format(frame_idx, segtracker.get_obj_num()), end='\r')
    print('\n-backward tracking finished')
